In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [7]:
#--export--#
from exports.e_07_Annealing import *

### Normalization

In [8]:
#--export--#
def normalize(x, m, s): return (x-m)/s

def normalize_data(train, valid):
    m,s = train.mean(),train.std()
    return normalize(train, m, s), normalize(valid, m, s)

In [23]:
#--export--#
def MNISTDataWrapper():
    x_train, y_train, x_valid, y_valid = loadMNIST()
    x_train, x_valid = normalize_data(x_train, x_valid)

    train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

    n_sampl, n_inp = x_train.shape
    n_out = 10
    n_hid = 50

    batch_size = 512

    return DataWrapper(*make_dls(train_ds, valid_ds, batch_size), n_out)

In [ ]:
data_w = MNISTDataWrapper()

# Lambda Layer
This will allow us to convert the MNIST data between square and flat formats, which will allow us to use a convnet.

In [24]:
#--export--#
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
        
    def forward(self, x): return self.func(x)
        
def flatten(x): return x.view(x.shape[0], -1)
def mnist_square(x): return x.view(-1 , 1, 28, 28)

In [25]:
#--export--#
def CNNModel(data_w, lr=0.3):
    n_inp, n_out = data_w.train_ds.x.shape[1], data_w.n_out
    
    model = nn.Sequential(
        Lambda(mnist_square),
        nn.Conv2d( 1, 8, 5, padding=2,stride=2), nn.ReLU(), # 14
        nn.Conv2d( 8,16, 3, padding=2,stride=2), nn.ReLU(), # 7
        nn.Conv2d(16,32, 3, padding=1,stride=2), nn.ReLU(), # 4
        nn.Conv2d(32,32, 3, padding=1,stride=2), nn.ReLU(), # 2
        nn.AdaptiveAvgPool2d(1),
        Lambda(flatten),
        nn.Linear(32,n_out)
    )
    
    return model, optim.SGD(model.parameters(), lr=lr)

In [26]:
model_w = ModelWrapper(*CNNModel(data_w), F.cross_entropy, data_w)
cbs = [AvgStatsCB([acc_f])]
job = DLJob(cbs)

## CNNs are much slower, but more accurate

In [27]:
%timeit job.fit(1, model_w)

train: [1.9277984375, tensor(0.3548)]
valid: [0.72882373046875, tensor(0.7561)]
train: [0.449520546875, tensor(0.8605)]
valid: [0.25144990234375, tensor(0.9257)]
train: [0.2117690625, tensor(0.9358)]
valid: [0.15630604248046875, tensor(0.9542)]
train: [0.1521771875, tensor(0.9543)]
valid: [0.1216452392578125, tensor(0.9636)]
train: [0.125634755859375, tensor(0.9612)]
valid: [0.11315538330078125, tensor(0.9665)]
train: [0.10199859375, tensor(0.9692)]
valid: [0.13249073486328125, tensor(0.9588)]
train: [0.091045458984375, tensor(0.9721)]
valid: [0.15326160888671875, tensor(0.9544)]
train: [0.0812145361328125, tensor(0.9752)]
valid: [0.0932900634765625, tensor(0.9714)]
3.14 s ± 345 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
!python utils/export_notebook.py 08_LambdaLayers.ipynb

Notebook 08_LambdaLayers.ipynb has been converted to module ./exports/e_08_LambdaLayers.py!
